# Stage 4: Matching
## Import required packages


In [1]:
# import py_entitymatching package
import py_entitymatching as em
import pandas as pd
import re
import nltk

/home/karan/anaconda2/envs/npy/lib/python2.7/site-packages/scikit_learn-0.18.1-py2.7-linux-x86_64.egg/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Read the input tables

In [2]:
# specify filepaths for tables A and B. 
path_A = 'spoj/json/spoj_blocking.csv' # patoh_A is the file path where table_A.csv is stored
path_B = 'codechef/data/codechef_blocking.csv' # path_B is the file path where table_B.csv is stored

In [3]:
# read table A; table A has 'ID' as the key attribute
A = em.read_csv_metadata(path_A)

# read table B; table B has 'ID' as the key attribute
B = em.read_csv_metadata(path_B)
A.rename(columns={"Unnamed: 0":"ID"}, inplace=True)
B.rename(columns={"Unnamed: 0":"ID"}, inplace=True)

No handlers could be found for logger "py_entitymatching.io.parsers"


In [4]:
A["words"] = A["description"].fillna("") + A["input"].fillna("") + A["output"].fillna("")
A = A.drop(["description", "input", "output"], axis=1)
B["words"] = B["description"].fillna("") + B["input"].fillna("") + B["output"].fillna("")
B = B.drop(["description", "input", "output"], axis=1)

In [5]:
# Step 1
def std_words(df):
    letters_only = re.sub("[^a-zA-Z]", " ", df["words"]) # letters only, drop numbers & symbols
    words = letters_only.lower().split() # lower case, split by word                  
    stops = set((nltk.corpus.stopwords.words("english") + 
                ['a','b','c','d','e','f','g','h','i','j','k','l','m','n',
                'o','p','q','r','s','t','u','v','w','x','y','z'])) # general stopwords & letters  
    meaningful_words = [w for w in words if not w in stops] # remove stopwords
    return( " ".join( meaningful_words)) # return re-joined string

# Step 2
A["words"] = A.apply(std_words, axis=1)
B["words"] = B.apply(std_words, axis=1)
em.set_key(A, "ID")
em.set_key(B, "ID")

True

## Read in candidate set

In [6]:
# read C
C = em.read_csv_metadata('candidate_set.csv', ltable=A, rtable=B)

In [7]:
table_A = pd.read_csv("spoj/json/problems.csv")
table_B = pd.read_csv("codechef/data/codechef_problems.csv")

In [8]:
C["ltable_url"] = table_A.loc[C.ltable_ID.values,"url"].values
C["rtable_url"] = table_B.loc[C.rtable_ID.values,"link"].values

In [9]:
C["ltable_problem_code"] = table_A.loc[C.ltable_ID.values,"title"].values
C["ltable_problem_code"] = table_A.loc[C.ltable_ID.values,"title"].apply(lambda x: x.split(" - ")[0]).values
C["rtable_problem_code"] = table_B.loc[C.rtable_ID.values,"problem_code"].values

In [10]:
C["ltable_problem_title"] = table_A.loc[C.ltable_ID.values,"title"].apply(lambda x:x.split(" - ")[1]).values
C["rtable_problem_title"] = table_B.loc[C.rtable_ID.values,"title"].values

In [11]:
D = C.sample(400, random_state=44)

In [12]:
D.head()

,_id,ltable_ID,rtable_ID,ltable_url,rtable_url,ltable_problem_code,rtable_problem_code,ltable_problem_title,rtable_problem_title
40378,40378,4204,4678,http://www.spoj.com/problems/GUESS003,https://www.codechef.com/problems/DS06,GUESS003,DS06,Guess the task III,Total Score
13861,13861,1404,4543,http://www.spoj.com/problems/FACTMUL,https://www.codechef.com/problems/IITK1P02,FACTMUL,IITK1P02,Product of factorials,Awesomeness of Strings
5325,5325,622,545,http://www.spoj.com/problems/TAP2012A,https://www.codechef.com/problems/FIN05,TAP2012A,FIN05,Awari 2,Laurel Creek
6912,6912,701,5305,http://www.spoj.com/problems/QN03,https://www.codechef.com/problems/SIMNIM,QN03,SIMNIM,Fan Switches,Simultaneous Nim
9663,9663,959,2247,http://www.spoj.com/problems/GO4DIMON,https://www.codechef.com/problems/LOC02,GO4DIMON,LOC02,GO FOR DIAMONDS,LOC Reality Show


In [13]:
D.loc[:,["ltable_url","ltable_problem_code","rtable_url","rtable_problem_code"]].to_csv('labelled_set.csv', index=False)